In [7]:
import torch

from torch.utils.data import DataLoader

from data.dataset_class import MultiLabelDataset
from base_model import ResNetModel
from run_train import run_epochs


In [2]:
base_model = ResNetModel()

In [3]:
model = base_model.get_model

In [4]:
train_data = DataLoader(MultiLabelDataset("train"), batch_size=64, shuffle=False)
val_data = DataLoader(MultiLabelDataset("val"), batch_size=64, shuffle=False)
test_data = DataLoader(MultiLabelDataset("test"), batch_size=64, shuffle=False)

In [5]:
history = run_epochs(1, train_data, val_data, model)

Epoch: [1/1], train_loos: 0.7649, train_acc: 0.9320, val_loos: 1.8762,  val_acc: 0.8929


In [12]:
example_data = MultiLabelDataset("test")

img, target = example_data[123]
model.eval()
pred = model(img.unsqueeze(0))

print("Target: ", target)
print("Predict", (torch.sigmoid(pred) > 0.5).float())

Target:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])
Predict tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.]])
